In [9]:
import csv
import numpy as np
import os
import scgt

In [10]:
# Reclassifies the terrain into [0, 1] transmission.

def dict_translate(np_arr, my_dict, default_val=0):
    """Translates the terrain type according to a dictionary mapping
    terrain type to values.
    """
    u,inv = np.unique(np_arr,return_inverse = True)
    return np.array([(my_dict.get(x, default_val)) for x in u])[inv].reshape(np_arr.shape)

def read_transmission_csv(fn):
    """Reads a dictionary of terrain resistance or transmission in csv, producing a dictionary."""
    d = {}
    with open(fn) as f:
        reader = csv.DictReader(f)
        for row in reader:
            if 'resistance' in row:
                d[int(row['map_code'])] = 1. - float(row['resistance'])
            else:
                d[int(row['map_code'])] = float(row['transmission'])
    return d



In [11]:
# Decide what you want to run.  You need to provide a bird (6-letter ebird nickname), and 
# whether to run on cropped territory or full. 
cropped = False
DATA_PATH = "."
layers = "CroppedLayers" if cropped else "OriginalLayers"

In [12]:
def gen_bird_layers(bird):
    INPUT_HABITAT = os.path.join(DATA_PATH, bird, layers, "habitat.tif")
    INPUT_TERRAIN = os.path.join(DATA_PATH, bird, layers, "terrain.tif")
    INPUT_TRANSMISSION_DICTIONARY = os.path.join(DATA_PATH, bird, "transmission_refined_1.csv")
    OUTPUT_TRANSMISSION = os.path.join(DATA_PATH, bird, layers, "transmission.tif")
    # Reads input layers.
    habitat_geotiff = scgt.GeoTiff.from_file(INPUT_HABITAT)
    terrain_geotiff = scgt.GeoTiff.from_file(INPUT_TERRAIN)
    habitat = habitat_geotiff.get_all_as_tile()
    terrain = terrain_geotiff.get_all_as_tile()
    # Translates into conductance.
    terr_to_transmission = read_transmission_csv(INPUT_TRANSMISSION_DICTIONARY)
    conductance = dict_translate(terrain.m, terr_to_transmission, default_val=0)
    # And conductance is 1 in the habitat (1 is the maximum value we use).
    conductance = np.maximum(conductance, habitat.m)
    # Rescales the conductance.
    conductance = 1 + 10 * conductance
    # Writes the result.
    with habitat_geotiff.clone_shape(OUTPUT_TRANSMISSION) as output_transmission:
        habitat.m = conductance
        output_transmission.set_tile(habitat)
    

In [13]:
for b in ["acowoo", "stejay"]:
    gen_bird_layers(b)